### Working with SMART on FHIR ID Tokens (examples)

In [1]:
# !pip3 install python-jose
from Crypto.PublicKey import RSA
import json
import jose.jwk
import jose.jwt
import jose.constants

### Setup
To create self-contained example, we'll generate a new RSA Key for a fake
organization called "my-ehr.org", and we'll use that for the operations below.

In [2]:
key = RSA.generate(2048)

private = key.exportKey('PEM').decode()
public = key.publickey().exportKey().decode()
print(public, "\n\n", private)

### Creating an ID Token (for servers)
Servers will create a signed JWT by following a process like this.

#### Create a set of claims
These should include:
 * `sub`: the user
 * `aud`: the app for whom this ID Token is being produced
 * `iss`: an identifier for this EHR system)
 * `profile`: the absolute URL of the FHIR resource representing the current user
 
#### Encode them in a JWT
Signing with the server's private key

In [3]:
claims = {
  "sub": "alice",
  "aud": "growth-chart-app-123",
  "iss": "https://my-ehr.org/fhir",
  "fhirUser": "https://my-ehr.org/fhir/Practitioner/123"
}


id_token = jose.jwt.encode(
    claims,
    key,
    algorithm='RS384')

print(id_token)

### Validating and using an ID Token (for clients)
A client obtains the ID Token as the result of an authorization operation. To validate the token, the client fetches the servers's public key, and then decodes the token. While decoding the token, the client must verify that the audience ('aud') matches its own client_id



In [4]:
jose.jwt.decode(id_token, public, audience='growth-chart-app-123')